In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 4s (1,133 kB/s)
debconf: unable to initi

In [4]:
!pip install facenet_pytorch==0.1.0
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 25.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 46.0 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
%cd /content/drive/MyDrive/driver_plate_identify

/content/drive/MyDrive/driver_plate_identify


In [ ]:
DRIVER_PATH = 'Driver_dataset'
MODEL_DIR_PATH = 'model'
MODEL_PATH = 'model/face_recogniser.pkl'

In [1]:
import os
from collections import namedtuple
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import transforms, datasets
from facenet_pytorch.models.utils.detect_face import extract_face
from PIL import Image, ImageDraw, ImageFont
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [ ]:
class FaceFeaturesExtractor:
    def __init__(self):
        self.aligner = MTCNN(prewhiten=False, keep_all=True, thresholds=[0.6, 0.7, 0.9])
        self.facenet_preprocess = transforms.Compose([Whitening()])
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval()

    def extract_features(self, img):
        bbs, _ = self.aligner.detect(img)
        if bbs is None:
            # if no face is detected
            return None, None

        faces = torch.stack([extract_face(img, bb) for bb in bbs])
        embeddings = self.facenet(self.facenet_preprocess(faces)).detach().numpy()

        return bbs, embeddings

    def __call__(self, img):
        return self.extract_features(img)

In [ ]:
exif_orientation_tag = 0x0112
exif_transpose_sequences = [  # Val  0th row  0th col
    [],  # 0    (reserved)
    [],  # 1   top      left
    [Image.FLIP_LEFT_RIGHT],  # 2   top      right
    [Image.ROTATE_180],  # 3   bottom   right
    [Image.FLIP_TOP_BOTTOM],  # 4   bottom   left
    [Image.FLIP_LEFT_RIGHT, Image.ROTATE_90],  # 5   left     top
    [Image.ROTATE_270],  # 6   right    top
    [Image.FLIP_TOP_BOTTOM, Image.ROTATE_90],  # 7   right    bottom
    [Image.ROTATE_90],  # 8   left     bottom
]


class ExifOrientationNormalize(object):
    """
    Normalizes rotation of the image based on exif orientation info (if exists.)
    """

    def __call__(self, img):
        if 'parsed_exif' in img.info and exif_orientation_tag in img.info['parsed_exif']:
            orientation = img.info['parsed_exif'][exif_orientation_tag]
            transposes = exif_transpose_sequences[orientation]
            for trans in transposes:
                img = img.transpose(trans)
        return img


class Whitening(object):
    """
    Whitens the image.
    """

    def __call__(self, img):
        mean = img.mean()
        std = img.std()
        std_adj = std.clamp(min=1.0 / (float(img.numel()) ** 0.5))
        y = (img - mean) / std_adj
        return y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: ROTATE_180 is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.ROTATE_180 instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: FLIP_TOP_BOTTOM is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_TOP_BOTTOM instead.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: ROTATE_90 is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.ROTATE_90 instea

In [ ]:
# Only some parts are being used

Prediction = namedtuple('Prediction', 'label confidence')
Face = namedtuple('Face', 'top_prediction bb all_predictions')
BoundingBox = namedtuple('BoundingBox', 'left top right bottom')


def top_prediction(idx_to_class, probs):
    top_label = probs.argmax()
    return Prediction(label=idx_to_class[top_label], confidence=probs[top_label])


def to_predictions(idx_to_class, probs):
    return [Prediction(label=idx_to_class[i], confidence=prob) for i, prob in enumerate(probs)]


class FaceRecogniser:
    def __init__(self, feature_extractor, classifier, idx_to_class):
        self.feature_extractor = feature_extractor
        self.classifier = classifier
        self.idx_to_class = idx_to_class

    def recognise_faces(self, img):
        bbs, embeddings = self.feature_extractor(img)
        if bbs is None:
            # if no faces are detected
            return []

        predictions = self.classifier.predict_proba(embeddings)

        return [
            Face(
                top_prediction=top_prediction(self.idx_to_class, probs),
                bb=BoundingBox(left=bb[0], top=bb[1], right=bb[2], bottom=bb[3]),
                all_predictions=to_predictions(self.idx_to_class, probs)
            )
            for bb, probs in zip(bbs, predictions)
        ]

    def __call__(self, img):
        return self.recognise_faces(img)

In [ ]:
def dataset_to_embeddings(dataset, features_extractor):
    transform = transforms.Compose([
        ExifOrientationNormalize(),
        transforms.Resize(1024)
    ])

    embeddings = []
    labels = []
    for img_path, label in dataset.samples:
        print(img_path)
        _, embedding = features_extractor(transform(Image.open(img_path).convert('RGB')))
        if embedding is None:
            print("Could not find face on {}".format(img_path))
            continue
        if embedding.shape[0] > 1:
            print("Multiple faces detected for {}, taking one with highest probability".format(img_path))
            embedding = embedding[0, :]
        embeddings.append(embedding.flatten())
        labels.append(label)

    return np.stack(embeddings), labels

In [ ]:
def load_data(features_extractor):
    dataset = datasets.ImageFolder(DRIVER_PATH)
    embeddings, labels = dataset_to_embeddings(dataset, features_extractor)
    return embeddings, labels, dataset.class_to_idx

In [ ]:
def train(embeddings, labels):
    softmax = LogisticRegression(solver='lbfgs', multi_class='multinomial', C=10, max_iter=10000)
    clf = softmax
    clf.fit(embeddings, labels)

    return clf

In [ ]:
def draw_bb_on_img(faces, img):
    draw = ImageDraw.Draw(img)
    fs = max(20, round(img.size[0] * img.size[1] * 0.000005))
    font = ImageFont.truetype('fonts/font.ttf', fs)
    margin = 5

    for face in faces:
        text = "%s %.2f%%" % (face.top_prediction.label.upper(), face.top_prediction.confidence * 100)
        text_size = font.getsize(text)

        # bounding box
        draw.rectangle(
            (
                (int(face.bb.left), int(face.bb.top)),
                (int(face.bb.right), int(face.bb.bottom))
            ),
            outline='green',
            width=2
        )

        # text background
        draw.rectangle(
            (
                (int(face.bb.left - margin), int(face.bb.bottom) + margin),
                (int(face.bb.left + text_size[0] + margin), int(face.bb.bottom) + text_size[1] + 3 * margin)
            ),
            fill='black'
        )

        # text
        draw.text(
            (int(face.bb.left), int(face.bb.bottom) + 2 * margin),
            text,
            font=font
        )
    return draw

In [ ]:
features_extractor = FaceFeaturesExtractor()
embeddings, labels, class_to_idx = load_data(features_extractor)
clf = train(embeddings, labels)

idx_to_class = {v: k for k, v in class_to_idx.items()}

target_names = map(lambda i: i[1], sorted(idx_to_class.items(), key=lambda i: i[0]))
print(metrics.classification_report(labels, clf.predict(embeddings), target_names=list(target_names)))

if not os.path.isdir(MODEL_DIR_PATH):
    os.mkdir(MODEL_DIR_PATH)
model_path = os.path.join('model', 'face_recogniser.pkl')
joblib.dump(FaceRecogniser(features_extractor, clf, idx_to_class), model_path)

Driver_dataset/Abdul_Hanan/Abdul Hanan.jpg
Driver_dataset/Abdullah_Anas/Abdullah Anas.jpg
Driver_dataset/Abdullah_Ramzan/Abdullah Ramzan.jpeg
Driver_dataset/Abdur_Rafay/Abdur Rafay.jpeg
Driver_dataset/Ahmad_Ashraf/Ahmad Ashraf.jpg
Driver_dataset/Aiza_Ali/Aiza Ali.jpeg
Driver_dataset/Amina_Habib/Amina Habib.jpeg
Driver_dataset/Amir_Arsalan/Amir Arsalan.jpg
Driver_dataset/Arsalan_Waqas/Arsalan Waqas.jpeg
Driver_dataset/Asghar_Ismail/Asghar Ismail.jpg
Driver_dataset/Asif_Ali/Asif Ali.jpg
Multiple faces detected for Driver_dataset/Asif_Ali/Asif Ali.jpg, taking one with highest probability
Driver_dataset/Asim_Raheel/Asim Raheel.jpg
Driver_dataset/Haroon_Yousaf/Haroon Yousaf.jpg
Driver_dataset/Hassan_Munir/Hassan Munir.jpeg
Driver_dataset/Iqra_Mubeen/Iqra Mubeen.jpeg
Driver_dataset/Misbah_Kiran/Misbah Kiran.jpeg
Driver_dataset/Momena_Khalil/Momena Khalil.jpeg
Driver_dataset/Momin_Shehzad/Momin Shehzad.jpg
Driver_dataset/Muaaz_Tehami/Muaaz Tehami.jpg
Driver_dataset/Muhammad_Anas/Muhammad Anas

['model/face_recogniser.pkl']

In [ ]:
image_path = '/content/drive/MyDrive/driver_plate_identify/Driver_dataset/Abdullah_Anas/Abdullah Anas.jpg'

preprocess = ExifOrientationNormalize()
img = Image.open(image_path)
filename = img.filename
img = preprocess(img)
img = img.convert('RGB')

faces = joblib.load(MODEL_PATH)(img)
if faces:
  print('The face is of: ', faces[0].top_prediction.label.upper())
  print('The prediction precentage is: ', faces[0].top_prediction.confidence * 100)

if not faces:
    print('No faces found in this image.')

The face is of:  ABDULLAH_ANAS
The prediction precentage is:  27.85427870798222


## Number plate detection

In [ ]:
import pytesseract
import cv2

In [ ]:
def cleanup_text(text):
	# strip out non-ASCII text so we can draw the text on the image
  # using OpenCV
  text = text.replace('\n', ' ')
  return "".join([c if ord(c) < 128 else "" for c in text]).strip()

In [ ]:
img = cv2.imread('/content/drive/MyDrive/driver_plate_identify/Number_Plate_dataset/Abdul Hanan.jpeg',cv2.IMREAD_COLOR)
text = pytesseract.image_to_string(img, config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 --psm 11')

In [ ]:
text

'ICT-ISLAMABAD\n\nAVS:822)\n\x0c'

In [ ]:
cleanup_text(text)

'ICT-ISLAMABAD  AVS:822)'